In [153]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [154]:
import numpy as np
import numpy.random as rnd
import numpy.linalg as la
import polars as pl
import pandas as pd
import datetime as dt
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import networkx as nx
import raphtory as rp
import umap
import community
import torch
import torch_geometric as tg
from torch_geometric.data import Data
from torch_geometric.utils.convert import from_networkx
from torch_geometric.transforms import LargestConnectedComponents
from torch_geometric.utils import to_networkx, from_networkx
from torch_geometric.nn import Node2Vec, GCNConv, VGAE
import torch.nn.functional as F

In [155]:
from datasets import DataLoader

# <font color="grey"> $\quad$ New Autonomous Systems dataset </font>

$\newcommand{\vct}[1]{\mathbf{#1}}$
$\newcommand{\mtx}[1]{\mathbf{#1}}$
$\newcommand{\e}{\varepsilon}$
$\newcommand{\norm}[1]{\|#1\|}$
$\newcommand{\minimize}{\mathrm{minimize}\quad}$
$\newcommand{\maximize}{\mathrm{maximize}\quad}$
$\newcommand{\subjto}{\quad\text{subject to}\quad}$
$\newcommand{\R}{\mathbb{R}}$
$\newcommand{\C}{\mathbb{C}}$
$\newcommand{\N}{\mathbb{N}}$
$\newcommand{\Z}{\mathbb{Z}}$
$\newcommand{Prob}{\mathbb{P}}$
$\newcommand{Expect}{\mathbb{E}}$
$\newcommand{Cov}{\mathrm{Cov}}$
$\newcommand{Var}{\mathrm{Var}}$
$\newcommand{\trans}{T}$
$\newcommand{\ip}[2]{\langle {#1}, {#2} \rangle}$
$\newcommand{\zerovct}{\vct{0}}$
$\newcommand{\diff}[1]{\mathrm{d}{#1}}$
$\newcommand{\conv}{\operatorname{conv}}$
$\newcommand{\inter}{{\operatorname{int}}}$

### <font color="grey">  Table of Contents</font>

1. #### <a href='#chapter1'>Data</a>
2. #### <a href='#chapter2'>Embedding</a>
3. #### <a href='#chapter3'>Visualisation</a>

###  <a id='chapter1'> <font color="grey">1. Data </font></a>

The data can be accessed via the dataloader. It is saved in the datasets/data/nas directory in two parquet files. 

In [81]:
dl = DataLoader(source='nAS')

In [82]:
# Get the edges
edge_df = dl.get_edges()
edge_df.head()

timestamp,source,dest,weight
datetime[μs],i64,i64,i64
2024-09-29 00:00:00,8151,1840,1
2024-09-29 00:00:00,8151,10420,1
2024-09-29 00:00:00,8151,136907,1
2024-09-29 00:00:00,8151,7173,1
2024-09-29 00:00:00,8151,28391,1


The weight columns has no relevance, it is always 1. 

In [83]:
# Get the nodes
node_df = dl.get_nodes()
node_df.head()

timestamp,nodes,country_code
datetime[μs],i64,i64
2024-09-29 00:00:00,8151,150
2024-09-29 00:00:00,1840,150
2024-09-29 00:00:00,25220,52
2024-09-29 00:00:00,198570,52
2024-09-29 00:00:00,4657,191


In [159]:
nodes = node_df['nodes'].unique().to_numpy()
len(nodes)

85069

The nodes have one feature: the country. This can be used to label and identify different subgraphs. In order to identify the features with a country, there is the country_code file.

In [160]:
country_codes = pl.read_parquet('./datasets/data/nas/country_codes.parquet')
country_codes[:2]

index,country
i64,str
0,"""AD"""
1,"""AE"""


In [85]:
# The timestamp column consists of datetime objects
date = node_df['timestamp'][0]
date

datetime.datetime(2024, 9, 29, 0, 0)

In [86]:
dates = dl.get_dates()
dates

[datetime.datetime(2024, 9, 29, 0, 0),
 datetime.datetime(2024, 9, 30, 0, 0),
 datetime.datetime(2024, 10, 1, 0, 0),
 datetime.datetime(2024, 10, 2, 0, 0),
 datetime.datetime(2024, 10, 3, 0, 0),
 datetime.datetime(2024, 10, 4, 0, 0),
 datetime.datetime(2024, 10, 5, 0, 0),
 datetime.datetime(2024, 10, 6, 0, 0),
 datetime.datetime(2024, 10, 7, 0, 0),
 datetime.datetime(2024, 10, 8, 0, 0),
 datetime.datetime(2024, 10, 9, 0, 0),
 datetime.datetime(2024, 10, 10, 0, 0),
 datetime.datetime(2024, 10, 11, 0, 0),
 datetime.datetime(2024, 10, 12, 0, 0),
 datetime.datetime(2024, 10, 13, 0, 0)]

In [168]:
# Finally, we can also get a graph in the torch-geometric format, which is useful for graph neural networks
data = dl.get_tgeometric()

100%|███████████████████████████████████████| 15/15 [00:04<00:00,  3.65it/s]


There may be a bug in this feature, will check later

A description of how pytorch-geometric deals with its graph data structure can be found [here](https://pytorch-geometric.readthedocs.io/en/latest/get_started/introduction.html).

In [169]:
data = data[dates[0]]

In [170]:
# Show the edges
data.edge_index

tensor([[  8151,   8151,   8151,  ..., 142340, 152037, 152037],
        [  1840,  10420, 136907,  ..., 142340, 152037, 150476]])

In [171]:
def speye(n, dtype=torch.float):
    """identity matrix of dimension n as sparse_coo_tensor."""
    return torch.sparse_coo_tensor(torch.tile(torch.arange(n, dtype=torch.long), (2, 1)),
                                   torch.ones(n, dtype=dtype),
                                   (n, n))

In [172]:
# Use empty features
# Use this as features in the absence of features
data.x = speye(len(nodes))

In [173]:
data.num_node_features

85069

###  <a id='chapter2'> <font color="grey">2. Embedding </font></a>

For the embedding, we use the architecture of a Variational Graph Autoencoder. Given a graph $G=(V,E)$ with $|V|=n$ nodes and node features $\vct{x}_i\in \R^d$, $i\in [n]$, denote by $\vct{X}=[\vct{x}_1,\dots,\vct{x}_n]^T\in \R^{n\times d}$ the features matrix and by $A=(a_{ij})\in \{0,1\}^{n\times n}$ the adjacency matrix of the graph. The **encoder** produces latent representations $\vct{z}_i\in \R^k$ for $i\in [n]$, which are sampled from the inference model
\begin{equation*}
  q(\vct{z}_i \ | \ \vct{X},\vct{A}) = \mathcal{N}(\vct{z}_i \ | \ \vct{\mu}_i,\mathrm{diag}(\vct{\sigma}_i)).
\end{equation*}
The means $\mu_i$ and variances $\mathrm{diag}(\vct{\sigma}_i)$ are parametrized using an encoder network, for example, a graph convolutional neural network (GCN). Denoting by $\vct{Z}=[\vct{z}_1,\dots,\vct{z}_n]^T$ the matrix of latent represenations and by $\vct{\mu}$ and $\vct{\sigma}$ the matrices representing the means and variances, we have
\begin{equation*}
  \vct{\mu} = \mathrm{GCN}_{\mu}(\vct{X},\vct{A}), \quad \quad \log \vct{\sigma} = \mathrm{GCN}_{\sigma}(\vct{X},\vct{A}).
\end{equation*}
The **generative model** is a distribution on the adjacency matrix,
\begin{equation*}
  p(\mtx{A}\ | \ \vct{Z}) = \prod_{i,j} p(a_{ij} \ | \ \vct{z}_i,\vct{z}_j).
\end{equation*}
It is convenient to use
\begin{equation*}
  p(a_{ij}=1 \ | \ \vct{z}_i,\vct{z}_j) = \sigma(\vct{z}_i^T\vct{z}_j),
\end{equation*}
where $\sigma$ is the logistic sigmoid. In order to train the model, we optimize the evidence lower bound
\begin{equation*}
  \mathcal{L} = \Expect_{q(\vct{Z}\ | \ \vct{X},\vct{A})}[\log p(\mtx{A}\ | \ \mtx{Z})]-\mathrm{D}_{\mathrm{KL}}(q(\mtx{Z}\ | \ \mtx{X},\mtx{A}) \ \| \ p(\mtx{Z})).
\end{equation*}

In [138]:
class Encoder(torch.nn.Module):
    """
    Implement a Graph Convolutional Network (GCN) as encoder
    """
    def __init__(self, dim, num_node_features, hidden_dim=32, cached=True, bias=True, add_self_loops=True, normalize=True):
        super().__init__()
        self.x_conv = tg.nn.GCNConv(num_node_features, 
                                    hidden_dim, 
                                    cached=cached, 
                                    bias=bias, 
                                    add_self_loops=add_self_loops,
                                    normalize=normalize)
        self.mean_conv = tg.nn.GCNConv(hidden_dim, 
                                       dim, 
                                       cached=cached,
                                       bias=bias, 
                                       add_self_loops=add_self_loops,
                                       normalize=normalize)
        self.var_conv = tg.nn.GCNConv(hidden_dim, 
                                      dim, 
                                      cached=cached, 
                                      bias=bias, add_self_loops=add_self_loops,
                                      normalize=normalize)

    def forward(self, data):
        x = data.x
        edge_index = data.edge_index

        x = self.x_conv(x, edge_index)
        x = F.relu(x)

        mu = self.mean_conv(x, edge_index)
        sigma = self.var_conv(x, edge_index)
        return mu, sigma

In [145]:
model = VGAE(encoder=Encoder(10, data.num_node_features))
#model = VGAE(encoder=VGAEconv(2, test_data.num_node_features))

In [140]:
def VGAE_loss(model, data):
    return model.recon_loss(model.encode(data), data.edge_index) + model.kl_loss() / data.num_nodes

In [148]:
def train(data, model, loss_fun, num_epochs=100, verbose=True, lr=0.01, logger=lambda loss: None):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    # optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    # schedule = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_epochs)
    for e in tqdm(range(num_epochs)):
        optimizer.zero_grad()
        loss = loss_fun(model, data)
        loss.backward()
        optimizer.step()
        logger(float(loss))
        if verbose:
            if not e % 20:
                print(f'epoch {e}: loss={loss.item()}')
        # schedule.step()
    return model

In [149]:
model = train(data, model, VGAE_loss, num_epochs=200)

  0%|          | 0/200 [00:00<?, ?it/s]

epoch 0: loss=2.833024740219116
epoch 20: loss=1.0688931941986084
epoch 40: loss=0.8776754140853882
epoch 60: loss=0.8293766379356384
epoch 80: loss=0.8050824999809265
epoch 100: loss=0.7938762903213501
epoch 120: loss=0.7871164083480835
epoch 140: loss=0.7822022438049316
epoch 160: loss=0.7768582105636597
epoch 180: loss=0.770989179611206


In [ ]:
def VGAE_patch_embeddings(patch_data, dim=2, hidden_dim=32, num_epochs=100, decoder=None, device='cpu', lr=0.01):
    patch_list = []
    models = []
    for patch in patch_data:
        if patch.x is None:
            patch.x = speye(patch.num_nodes)
            print(f"training patch with {patch.edge_index.shape[1]} edges")   #added [i] to every patch
            model = tg.nn.VGAE(encoder=VGAEconv(dim, patch.x.shape[1], hidden_dim=hidden_dim), decoder=decoder).to(device)
            patch.to(device)

        model = train(patch, model, VGAE_loss, num_epochs=num_epochs, lr=lr)
        with torch.no_grad():
            model.eval()
            coordinates = model.encode(patch).to('cpu').numpy()
            models.append(model)
            patch_list.append(l2g.Patch(patch.nodes.to('cpu').numpy(), coordinates))
    return patch_list, models

###  <a id='chapter3'> <font color="grey">3. Visualisation </font></a>

In [ ]:
# Use UMAP to visualise the graph embeddings for different days